In [ ]:
import struct
import ctypes
import os
import matplotlib.pyplot as plt

: 

In [ ]:
# to do add a python function to read a matrix of binaries
def ReadBinaryLog(contents, offset_btye, binary_format, formatSize, num_of_cols, num_of_rows):
    data = list()
    for i in range(num_of_cols):
        data.append(list())

    stateSize =  num_of_cols * formatSize
    format_sq = num_of_cols * binary_format
    for i in range(num_of_rows):
        start_idx = offset_btye + i * stateSize
        end_idx = offset_btye + (i + 1) * stateSize
        var = struct.unpack(format_sq, contents[start_idx: end_idx])
        for j in range(num_of_cols):
            data[j].append(var[j])

    return data

# add a python function to extract size information and save to dict
def ReadBinaryInfoInt(contents, offset_bytes, binary_format,
                   binary_format_size, num_of_data, name=None):
    data = ReadBinaryLog(contents, offset_bytes, binary_format, binary_format_size, num_of_data, 1)
    res = dict()
    for i in range(num_of_data):
        if name != None:
            res[name[i]] = int(data[i][0])
        else:
            res["unnamed_entry_" + str(i)] = int(data[i][0])
    return res


def ScanBinaryLog(filename, numOfColumns, format, formatSize):
    with open(filename, mode='rb') as f:
        contents = f.read()
    f.close()
    res = list()
    format_sq = numOfColumns * format
    stateSize = numOfColumns * formatSize
    totalSizeByte = len(contents)
    if (totalSizeByte % stateSize) != 0:
        raise Exception("size of " + filename + " is " + str(totalSizeByte) +
                        ", which is not a multiple of stateSize " + str(stateSize) + "!")

    numOfSamples = int(totalSizeByte / stateSize)

    for _ in range(numOfColumns):
        res.append(list())

    for i in range(numOfSamples):
        var = struct.unpack(format_sq, contents[(
            i * stateSize):((i + 1) * stateSize)])
        for j in range(numOfColumns):
            res[j].append(var[j])

    return res

In [ ]:
filename = "/home/zilin/Documents/UofT/beta_fly_sim/build_release/betaflight_test.dat"
with open(filename, mode='rb') as f:
    contents = f.read()
f.close()
double_len = ctypes.sizeof(ctypes.c_double)
info = ReadBinaryInfoInt(contents, offset_bytes=0, binary_format='d',
                         binary_format_size=double_len, num_of_data=2,
                         name=["time_step", "num_of_data"])

print(info)
data = ReadBinaryLog(contents, 2 * double_len, 'd', 
                     double_len,
                     num_of_cols=info['num_of_data'],
                     num_of_rows=info['time_step'])

In [ ]:
plt.figure(figsize=(10,10), dpi= 100)
plt.subplot(311)
plt.plot(data[0], data[1])
plt.ylabel('p(rad/s)')
plt.xlabel('t(s)')
plt.grid(True)
plt.subplot(312)
plt.plot(data[0], data[2])
plt.ylabel('q(rad/s)')
plt.xlabel('t(s)')
plt.grid(True)
plt.subplot(313)
plt.plot(data[0], data[3])
plt.ylabel('r(rad/s)')
plt.xlabel('t(s)')
plt.grid(True)



plt.figure(figsize=(10,10), dpi= 100)
plt.subplot(311)
plt.plot(data[0], data[8])
plt.ylabel('torque x (Nm)')
plt.xlabel('t(s)')
plt.grid(True)
plt.subplot(312)
plt.plot(data[0], data[9])
plt.ylabel('torque y (Nm)')
plt.xlabel('t(s)')
plt.grid(True)
plt.subplot(313)
plt.plot(data[0], data[10])
plt.ylabel('torque z (Nm)')
plt.xlabel('t(s)')
plt.grid(True)


plt.figure(figsize=(10,10), dpi= 100)
plt.plot(data[0], data[4])
plt.plot(data[0], data[5])
plt.plot(data[0], data[6])
plt.plot(data[0], data[7])
plt.legend(["r_0", "r_1", "r_2", "r_3"])
plt.ylabel('rpm')
plt.xlabel('t(s)')
plt.grid(True)